# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - https://github.com/tcbegley/cube-solver</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [VISUALIZE](#4)
* [SOLVE](#5)

Code modified from: https://www.kaggle.com/code/paulorzp/magic-cube-utilities/notebook

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [2]:
import os
import time


import random
from typing import Dict, List
import zipfile
import sqlite3

import magiccube

import numpy as np
import pandas as pd
from enum import Enum
from math import sqrt
from ast import literal_eval
from termcolor import colored
from sympy.combinatorics import Permutation

from twophase import solve

In [2]:
database_file = '../solutions.db'
solution_method = 'cube-solver'


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [11]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [69]:
tb = [['A','y','u'],
      ['B','g','f'],
      ['C','o','r'],
      ['D','b','b'],
      ['E','r','l'],
      ['F','w','d']]

# tb = [['A','y','u'],
#       ['B','g','r'],
#       ['C','o','f'],
#       ['D','b','d'],
#       ['E','r','l'],
#       ['F','w','b']]

def a2color(state):
    for c in tb:
        state = state.replace(c[0],c[1])
    return state.upper().replace(";","")

def a2face(state):
    for c in tb:
        state = state.replace(c[0],c[2])
    return state.upper().replace(";","")

def color2a(state):
    for c in tb:
        state = state.replace(c[1],c[0])
    return state.upper().replace(";","")

def face2a(state):
    for c in tb:
        state = state.replace(c[2],c[0])
    return state.upper().replace(";","")

In [70]:
def swap(state):
    state = state.replace(";","")
    s = int(sqrt((len(state) / 6)))
    init_state = []
    init_state.extend(list(state))
    foo = []
    foo.extend(init_state[0 : s*s])
    foo.extend(init_state[(s*s * 4) : (s*s * 5)])
    foo.extend(init_state[(s*s * 1) : (s*s * 2)])
    foo.extend(init_state[(s*s * 2) : (s*s * 3)])
    foo.extend(init_state[(s*s * 3) : (s*s * 4)])
    foo.extend(init_state[(s*s * 5) : (s*s * 6)])
    state =";".join(foo)
    return state

In [71]:
def reversed_moves(sol, schar):
    return f"{schar}".join(str(c) for c in reversed(sol.split(schar)))

def convert_moves(sol,s,reverse=False):
    moves = []
    for n in range(1,s-1):
        moves.extend([[f"{n+1}L ",f"-r{n}."],[f"{n+1}L' ",f"r{n}."],
                      [f"{n+1}F' ",f"-f{n}."],[f"{n+1}F ",f"f{n}."],
                      [f"{n+1}U ",f"-d{n}."],[f"{n+1}U' ",f"d{n}."]])
    moves.extend([
         ["F' ","-f0."],["F ","f0."],
         ["B ",f"-f{s-1}."],["B' ",f"f{s-1}."],
         ["R' ","-r0."],["R ","r0."],
         ["L ",f"-r{s-1}."],["L' ",f"r{s-1}."],
         ["D' ","-d0."],["D ","d0."],
         ["U ",f"-d{s-1}."],["U' ",f"d{s-1}."],
         ["F2 ","f0.f0."],["B2 ",f"f{s-1}.f{s-1}."],["R2 ","r0.r0."],
         ["L2 ",f"r{s-1}.r{s-1}."],["D2 ","d0.d0."],["U2 ",f"d{s-1}.d{s-1}."]
        ])
    
    if (sum(1 for c in sol if c.isupper())) > 1:
        schar = "."
        sol += " "
        for m in moves:
            sol = sol.replace(m[0],m[1])
        sol = sol.replace("..",".")
        sol = sol[:-1]
    else:
        schar = " "
        sol += "."
        for m in moves:
            sol = sol.replace(m[1],m[0])
    sol = sol.replace("  "," ")
    sol = sol.strip()
    if reverse:
        sol = reversed_moves(sol, schar)
    return sol

In [72]:
def move(state, allmoves, mv):
    power = 1
    if mv[0] == "-":
        mv = mv[1:]
        power = -1
    p = allmoves[mv]
    state = (p ** power)(state)
    return state

def run_moves(state, allmoves, moves):
    state = state.split(";")
    for m in moves.split("."):
        state = move(state, allmoves, m)
    state = ";".join(state)
    return state

In [73]:
colors = {' Y ': colored(' Y ',color='black', on_color='on_yellow'),
          ' W ': colored(' W ',color='black', on_color='on_white'),
          ' G ': colored(' G ',color='black', on_color='on_green'),
          ' B ': colored(' B ',color='black', on_color='on_light_blue'),
          ' R ': colored(' R ',color='black', on_color='on_light_red'),
          ' O ': colored(' O ',color='black', on_color='on_red')}

def set_colors(face):
    for key, value in colors.items():
        face = face.replace(key, value)
    return face

def cube2_2d(c):
    print(set_colors(f'       {c[0]}  {c[1]} '))
    print(set_colors(f'       {c[2]}  {c[3]} '))
    print(set_colors(f' {c[16]}  {c[17]}  {c[ 4]}  {c[ 5]}  {c[ 8]}  {c[ 9]}  {c[12]}  {c[13]} '))
    print(set_colors(f' {c[18]}  {c[19]}  {c[ 6]}  {c[ 7]}  {c[10]}  {c[11]}  {c[14]}  {c[15]} '))
    print(set_colors(f'       {c[20]}  {c[21]} '))
    print(set_colors(f'       {c[22]}  {c[23]} '))
    
def cube3_2d(c):
    print(set_colors(f'          {c[0]}  {c[1]}  {c[2]} '))
    print(set_colors(f'          {c[3]}  {c[4]}  {c[5]} '))
    print(set_colors(f'          {c[6]}  {c[7]}  {c[8]} '))
    print(set_colors(f' {c[36]}  {c[37]}  {c[38]}  {c[ 9]}  {c[10]}  {c[11]}  {c[18]}  {c[19]}  {c[20]}  {c[27]}  {c[28]}  {c[29]} '))
    print(set_colors(f' {c[39]}  {c[40]}  {c[41]}  {c[12]}  {c[13]}  {c[14]}  {c[21]}  {c[22]}  {c[23]}  {c[30]}  {c[31]}  {c[32]} '))
    print(set_colors(f' {c[42]}  {c[43]}  {c[44]}  {c[15]}  {c[16]}  {c[17]}  {c[24]}  {c[25]}  {c[26]}  {c[32]}  {c[34]}  {c[35]} '))
    print(set_colors(f'          {c[45]}  {c[46]}  {c[47]} '))
    print(set_colors(f'          {c[48]}  {c[49]}  {c[50]} '))
    print(set_colors(f'          {c[51]}  {c[52]}  {c[53]} '))
    
def print_cube(cube):
    print(set_colors(cube.__str__()))

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; 
text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >Visualize<br><div> 

<a id="5"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [74]:
N = 2
puzzle_id = 0
sol = puzzles.loc[puzzle_id]
state = sol['initial_state']
solut = sol['solution_state']
sub = submission.loc[puzzle_id]
allowed_moves = literal_eval(puzzle_info.loc[sol.puzzle_type, 'allowed_moves'])
allowed_moves = {k: Permutation(v) for k, v in allowed_moves.items()}
sol.puzzle_type,allowed_moves

('cube_2/2/2',
 {'f0': Permutation(23)(2, 19, 21, 8)(3, 17, 20, 10)(4, 6, 7, 5),
  'f1': Permutation(0, 18, 23, 9)(1, 16, 22, 11)(12, 13, 15, 14),
  'r0': Permutation(1, 5, 21, 14)(3, 7, 23, 12)(8, 10, 11, 9),
  'r1': Permutation(23)(0, 4, 20, 15)(2, 6, 22, 13)(16, 17, 19, 18),
  'd0': Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21),
  'd1': Permutation(23)(0, 1, 3, 2)(4, 16, 12, 8)(5, 17, 13, 9)})

In [75]:
print(state)
assert solut == run_moves(state, allowed_moves, sub.moves)
cube2_2d(a2color(state))
print(" ")
cube2_2d(a2color(run_moves(state, allowed_moves, sub.moves)))

D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C
       B  R 
       B  Y 
 W  W  R  G  O  Y  B  O 
 R  R  Y  G  O  Y  B  W 
       G  W 
       G  O 
 
       Y  Y 
       Y  Y 
 R  R  G  G  O  O  B  B 
 R  R  G  G  O  O  B  B 
       W  W 
       W  W 


In [76]:
cube = magiccube.Cube(N, a2color(swap(state)))
print_cube(cube)
cube.rotate(convert_moves(sub.moves,N))
print_cube(cube)

       B  R             
       B  Y             
 W  W  R  G  O  Y  B  O 
 R  R  Y  G  O  Y  B  W 
       G  W             
       G  O             

       Y  Y             
       Y  Y             
 R  R  G  G  O  O  B  B 
 R  R  G  G  O  O  B  B 
       W  W             
       W  W             


In [113]:
N = 3
puzzle_id = 32
sol = puzzles.loc[puzzle_id]
state = sol['initial_state']
solut = sol['solution_state']
sub = submission.loc[puzzle_id]
allowed_moves = literal_eval(puzzle_info.loc[sol.puzzle_type, 'allowed_moves'])
allowed_moves = {k: Permutation(v) for k, v in allowed_moves.items()}
# sol.puzzle_type,allowed_moves

layout = solut.split(';')
stickers = 54
dim = 3
face_size = dim**2
state_list = (
        layout[stickers-dim:stickers] + layout[stickers-2*dim:stickers-dim] + layout[stickers-3*dim:stickers-2*dim] +
layout[face_size+2*dim:face_size+3*dim] + layout[face_size + dim:face_size + 2*dim] + layout[face_size:face_size + dim]  +
layout[2*face_size+2*dim:2*face_size+3*dim] + layout[2*face_size + dim:2*face_size + 2*dim] + layout[2*face_size:2*face_size + dim] +
layout[3*face_size+2*dim:3*face_size+3*dim] + layout[3*face_size + dim:3*face_size + 2*dim] + layout[3*face_size:3*face_size + dim] +
layout[4*face_size+2*dim:4*face_size+3*dim] + layout[4*face_size + dim:4*face_size + 2*dim] + layout[4*face_size:4*face_size + dim] +
layout[2*dim:3*dim] + layout[dim:2*dim] + layout[:dim] )

translated_state = state_list[0:9] + state_list[18:27] + state_list[9:18] + state_list[45:54]  + state_list[36:45] + state_list[27:36]

sm = solve(a2face(';'.join(translated_state)))

# print(state_list)
# print(a2face(';'.join(layout)))

In [117]:
sm.replace('R2', 'r0 r0').replace('F2', 'f0 f0').replace('B2', 'f1 f1').replace('L2', 'r1 r1').split(' ')

['r0',
 'r0',
 'f0',
 'f0',
 'r0',
 'r0',
 'f0',
 'f0',
 'f1',
 'f1',
 'r0',
 'r0',
 'f0',
 'f0',
 'r1',
 'r1']

In [67]:
print(state)
print(swap(state))

F;B;B;E;E;D;B;E;E;F;A;D;C;D;A;B;E;A;F;C;A;C;F;F;D;A;B;E;C;D;E;B;D;C;F;A;C;B;E;A;A;F;E;F;C;A;D;C;D;C;B;D;B;F
F;B;B;E;E;D;B;E;E;C;B;E;A;A;F;E;F;C;F;A;D;C;D;A;B;E;A;F;C;A;C;F;F;D;A;B;E;C;D;E;B;D;C;F;A;A;D;C;D;C;B;D;B;F


In [105]:
stickers = 54
dim = 3
face_size = dim**2
layout = list(range(stickers))
(layout[stickers-dim:stickers] + layout[stickers-2*dim:stickers-dim] + layout[stickers-3*dim:stickers-dim] +
layout[face_size+2*dim:face_size+3*dim] + layout[face_size + dim:face_size + 2*dim] + layout[face_size:face_size + dim]  +
layout[2*face_size+2*dim:2*face_size+3*dim] + layout[2*face_size + dim:2*face_size + 2*dim] + layout[2*face_size:2*face_size + dim] +
layout[3*face_size+2*dim:3*face_size+3*dim] + layout[3*face_size + dim:3*face_size + 2*dim] + layout[3*face_size:3*face_size + dim] +
layout[4*face_size+2*dim:4*face_size+3*dim] + layout[4*face_size + dim:4*face_size + 2*dim] + layout[4*face_size:4*face_size + dim] +
layout[2*dim:3*dim] + layout[dim:2*dim] + layout[:dim] )


[51,
 52,
 53,
 48,
 49,
 50,
 45,
 46,
 47,
 48,
 49,
 50,
 15,
 16,
 17,
 12,
 13,
 14,
 9,
 10,
 11,
 24,
 25,
 26,
 21,
 22,
 23,
 18,
 19,
 20,
 33,
 34,
 35,
 30,
 31,
 32,
 27,
 28,
 29,
 42,
 43,
 44,
 39,
 40,
 41,
 36,
 37,
 38,
 6,
 7,
 8,
 3,
 4,
 5,
 0,
 1,
 2]

In [68]:
';'.join(state_list)

'F;B;B;E;E;D;B;E;E;F;A;D;C;D;A;B;E;A;F;C;A;C;F;F;D;A;B;E;C;D;E;B;D;C;F;A;C;B;E;A;A;F;E;F;C;A;D;C;D;C;B;D;B;F'